In [64]:
import numpy as np
import cv2
import time
import math

WIND_X = 700
WIND_Y = 500

WIND_NAME = "Ilya MV survival game..."

cv2.namedWindow (WIND_NAME, cv2.WINDOW_NORMAL)
cv2.resizeWindow (WIND_NAME, (WIND_X, WIND_Y))

class Object:
    def __init__ (self, x_, y_, vx_, vy_, ax_ = 0, ay_ = 0, name_ = "noname",
                  szx_ = 0, szy_ = 0, img_path_ = ""):
        self.x = x_
        self.y = y_
        self.vx = vx_
        self.vy = vy_
        self.ax = ax_
        self.ay = ay_
        self.szx = szx_
        self.szy = szy_

        self.name = name_
        self.img_path = img_path_
        
        if (self.img_path != ""):
            self.img = cv2.resize (cv2.imread (self.img_path), (self.szx, self.szy))
        
        else:
            self.img = np.ones ((self.szx, self.szy, 3), np.uint8) * 69
    
    def draw (self, canvas):
        canvas [self.y : self.y + self.szy, self.x : self.x + self.szx] = self.img
    
    def handle_kb (self, key):
        pass
    
    def move (self):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x <= 0 or self.x >= WIND_X - self.szx):
            self.vx *= -1
            self.x += self.vx

        if (self.y <= 0 or self.y >= WIND_Y - self.szy):
            self.vy *= -1
            self.y += self.vy
        
        self.vx += self.ax
        self.vy += self.ay
    
    def distance (self, obj):
        return math.sqrt ((self.x - obj.x)**2 + (self.y - obj.y)**2)

class Target (Object):
    def __init__ (self, x_, y_, vx_, vy_, ax_ = 0, ay_ = 0, name_ = "noname",
                  img_path_ = "", szx_ = 60, szy_ = 60, points_ = 3, lifetime_ = 10):
        Object.__init__ (self, x_, y_, vx_, vy_, ax_, ay_, name_, szx_, szy_, img_path_)
        
        self.creation_time = time.time ()
        self.points = points_
        self.lifetime = lifetime_
        
    def draw (self, canvas):
        canvas [self.y : self.y + self.szy, self.x : self.x + self.szx] = self.img
        
        cv2.putText (canvas, str (self.lifetime + self.creation_time - time.time ()) [:5],
            (self.x + 5, self.y + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 25, 130), 2, cv2.LINE_AA)

class Player (Object):
    def __init__ (self, x_, y_, vx_, vy_, ax_ = 0, ay_ = 0, name_ = "noname",
                  img_path_ = "", szx_ = 90, szy_ = 90):
        Object.__init__ (self, x_, y_, vx_, vy_, ax_, ay_, name_, szx_, szy_, img_path_)
        
    def handle_kb (self, key):
        if (key == ord ("i")):
            self.vy -= 1
        
        if (key == ord ("k")):
            self.vy += 1
    
        if (key == ord ("j")):
            self.vx -= 1
        
        if (key == ord ("l")):
            self.vx += 1
    
class Game:
    def __init__ (self, scenario_):
        self.scenario = scenario_
        
        self.starting_time = time.time ()
        self.last_adding_time = self.starting_time
        
        self.player = Player (10, 10, self.scenario ["player speed"],
            self.scenario ["player speed"], img_path_ = self.scenario ["player img"])
                
        self.objects = []
        
        for img_path in self.scenario ["target imgs"]:
            points = 3
            
            if ("2" in img_path):
                points = -5
            
            new_target = self.object_by_path (img_path, points)
            
            self.add_object (new_target)
        
        self.status = "playing"
        self.score = 0
    
    def add_random_object (self):
        target_num = np.random.randint (len (self.scenario ["target imgs"]))
        target_path = self.scenario ["target imgs"] [target_num]
        
        points = np.random.randint (2, 5)
        
        if ("2" in target_path):
            points *= -1
        
        obj = self.object_by_path (target_path, points)
        self.add_object (obj)
    
    def object_by_path (self, path, points):
        return Target (np.random.randint (WIND_X // 5, int (WIND_X * 4 / 5)),
                       np.random.randint (WIND_Y // 5, int (WIND_Y * 4 / 5)),
                       np.random.randint (2, 5), np.random.randint (2, 5),
                       img_path_ = path, points_ = points)
    
    def add_object (self, new_obj):
        self.objects.append (new_obj)
    
    def draw (self, canvas):
        self.player.draw (canvas)
        
        for obj in self.objects:
            obj.draw (canvas)
        
        cv2.putText (canvas, "Score: " + str (self.score), (30, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 25, 130), 2, cv2.LINE_AA)

        cv2.putText (canvas, "Time: " + str (self.scenario ["duration"] +
            self.starting_time - time.time ()) [:5], (30, 80),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 25, 130), 2, cv2.LINE_AA)
    
    def handle_collisions (self):
        for obj in self.objects:
            if (self.player.distance (obj) < self.scenario ["eating distance"]):
                self.score += obj.points
                self.objects.remove (obj)
                break
    
    def move (self):
        self.player.move ()
        
        for obj in self.objects:
            obj.move ()
    
    def handle_kb (self, key):
        self.player.handle_kb (key)
        
        for obj in self.objects:
            obj.handle_kb (key)
    
    def remove_old (self):
        for obj in self.objects:
            if (time.time () - obj.creation_time > self.scenario ["target lifetime"]):
                self.objects.remove (obj)
    
    def add_new (self):
        if (- self.last_adding_time + time.time () > self.scenario ["target spawn delay"]):
            self.add_random_object ()
            self.last_adding_time = time.time ()
    
    def game_iteration (self, time):
        self.remove_old ()
        self.add_new ()
        
        self.move ()
        self.handle_collisions ()
        
        if (self.score < 0):
            self.status = "lose"
        
        if (time.time () - self.starting_time > self.scenario ["duration"]):
            self.status = "win"

class Manager:
    def __init__ (self, config_):
        self.config = config_
        self.game = Game (self.config)
        self.status = "waiting"
        
    def handle_kb (self, key):
        if (self.status == "waiting"):
            if (key == ord ("n")):
                self.status = "playing"
                self.game = Game (self.config)

        if (self.status == "playing"):
            self.game.handle_kb (key)
        
        if (self.status == "finished"):
            if (key == ord ("c")):
                self.status = "waiting"
            
    def draw (self, canvas):
        if (self.status == "waiting"):
            cv2.putText (canvas, "Press n to play, ijkl - control", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 250, 130), 2, cv2.LINE_AA)
        
        if (self.status == "playing"):
            self.game.draw (canvas)
        
        if (self.status == "finished"):
            cv2.putText (canvas, self.game.status + " with " + str (self.game.score) +
                "points, press c", (30, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1, (100, 250, 130), 2, cv2.LINE_AA)
    
    def on_idle (self, time):
        if (self.game.status in ["win", "lose"]):
            self.status = "finished"
            self.game.status += " "
        
        if (self.status == "playing"):
            self.game.game_iteration (time)

game_config = {"player speed"       : 2,
               "duration"           : 60,
               "target spawn delay" : 2,
               "player img"         : "data/player.png",
               
               "target imgs"        : ["data/target1.png",
                                       "data/target2.png",
                                       "data/target3.png"],
               
               "target lifetime"    : 10,
               "eating distance"    : 40}

manager = Manager (game_config)

while (True):
    canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 200
    t      = time.time ()
    key    = cv2.waitKey (1) & 0xFF
    
    manager.handle_kb (key)
    manager.on_idle   (time)
    manager.draw      (canvas)

    cv2.imshow (WIND_NAME, canvas)
    time.sleep (0.01)

    if (key == ord('q')):
        break

cv2.destroyAllWindows()